In [60]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets

# --- Load CSV ---
df = pd.read_csv(
    r"C:\Users\Owner\OneDrive\Desktop\WEDLOOP\Patent-pub-tech-count2-byorigin-y99-y23.csv",
    skiprows=1,
    na_values=["N/A", "NA", "Na", "n/a"]
)
df = df.fillna(0)

# --- Melt the dataframe ---
year_columns = [col for col in df.columns if col.startswith("YR")]
df_melted = df.melt(
    id_vars=["Field of technology"], 
    value_vars=year_columns,
    var_name="Year",
    value_name="Patent_Count"
)

# Clean Year and convert counts
df_melted['Year'] = df_melted['Year'].str.replace("YR", "").astype(int)
df_melted['Patent_Count'] = (
    df_melted['Patent_Count'].astype(str)
    .str.replace(",", "")
    .str.strip()
)
df_melted['Patent_Count'] = pd.to_numeric(df_melted['Patent_Count'], errors='coerce').fillna(0)

# --- Function to plot interactive heatmap ---
def plot_heatmap(selected_years, selected_tech):
    # Filter technologies
    filtered_df = df_melted.copy()
    if selected_tech:
        filtered_df = filtered_df[filtered_df['Field of technology'].isin(selected_tech)]
    
    if selected_years:
        filtered_df = filtered_df[filtered_df['Year'].isin(selected_years)]

    # Pivot table
    heatmap_data = filtered_df.pivot_table(
        index="Field of technology",
        columns="Year",
        values="Patent_Count",
        aggfunc='sum',
        fill_value=0
    )

    # Normalize per technology
    heatmap_data_norm = heatmap_data.div(heatmap_data.max(axis=1), axis=0)

    # Plot
    plt.figure(figsize=(14, max(6, 0.5*len(heatmap_data_norm))))
    sns.heatmap(
        heatmap_data_norm,
        cmap="viridis",
        cbar_kws={'label': 'Normalized Patent Counts'},
        linewidths=0.3,
        annot=True,
        fmt=".2f"
    )
    plt.gca().xaxis.tick_top()
    plt.xticks(rotation=0)
    plt.title("Patent Publications by Technology (Normalized)", fontsize=16)
    plt.xlabel("Year")
    plt.ylabel("Technology")
    plt.show()

# --- Interactive widgets ---
all_years = sorted(df_melted['Year'].unique())
all_tech = sorted(df_melted['Field of technology'].unique())

interact(
    plot_heatmap,
    selected_years=widgets.SelectMultiple(
        options=all_years,
        value=(1999, 2005, 2010, 2015, 2020, 2023),
        description='Years'
    ),
    selected_tech=widgets.SelectMultiple(
        options=all_tech,
        value=(),
        description='Technologies'
    )
)


interactive(children=(SelectMultiple(description='Years', index=(0, 6, 11, 16, 21, 24), options=(1999, 2000, 2…

<function __main__.plot_heatmap(selected_years, selected_tech)>